
# Oh bien la montre !
Ici nous faire une demo du scraping de données sur amazon
Le but étant de trouvé des acticle classé par ordre de prix pour le moment 

In [1]:
from bs4 import BeautifulSoup
from urllib import request

Nous allons d'abors charger la page amazon qui contient tout les prooduits...


In [2]:
pattern_produit="camera+vlog"
url = f"https://www.amazon.fr/s?k={pattern_produit}&i=electronics"


def getPage(url):
    request_text = request.urlopen(url).read()
    return BeautifulSoup(request_text, 'lxml')

Nous pouvons afficher les produits par titre de la page sans le contenu sponsorisé


In [3]:
def getMytitleProduct(url):
    page = getPage(url)
    for item in page.find('div', {'class' : 's-main-slot s-result-list s-search-results sg-row'}).findAll({'div'}) : 
        if item.get("data-component-type") :
            title = item.find('a', {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'}).text.strip()[:25]
            print(title,"\n-------")

getMytitleProduct(url)

Caméscope 4K Caméra Vidéo 
-------
Sony ZV-1 | Kit Appareil  
-------
Acoletty Appareil Photo N 
-------
Sony ZV-1 | Appareil Vlog 
-------
KANDAO QooCam 8K Enterpri 
-------
NBD Vlogging Camera Appar 
-------
Caméscope 4K Caméra Vidéo 
-------
Sony Alpha ZV-E10 | Appar 
-------
ORDRO EP6 Plus Caméscope  
-------
NBD Appareil Photo numéri 
-------
Appareil Photo Numérique  
-------
Caméscope ORDRO EP8 4k 60 
-------
DJI Pocket 2 - Caméra 4K  
-------
Panasonic Lumix S5K | App 
-------
ORDRO 4K Caméscope Caméra 
-------
Panasonic Lumix GH5S | Ap 
-------
10" LED Ring Light avec E 
-------
Caméra Vidéo 2.7K Camésco 
-------
PNJ – CAMÉRA 4K Pocket -  
-------
Panasonic Lumix S5K | App 
-------
Téléprompteur 13 Pouces,  
-------
AKASO Caméra Sport 4K 30f 
-------
Sony RX100 III | Appareil 
-------
T osuny Appareil Photo Nu 
-------
